In [1]:
#hidden cell to be executed BEFORE the presentation
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import dftpy
from dftpy.ions import Ions
from dftpy.field import DirectField
from dftpy.grid import DirectGrid
from dftpy.functional import LocalPseudo, Functional, TotalFunctional
from dftpy.formats import io
from dftpy.math_utils import ecut2nr
from dftpy.time_data import TimeData
from dftpy.optimization import Optimization
from dftpy.mpi import sprint
from IPython.lib.display import YouTubeVideo
from IPython.display import IFrame
from ase.visualize import view
PP_list = {'Al': 'Al_lda.oe01.recpot'}
#import fortecubeview

<div id="bg-slide">
    <center>
    <h1>Introduction to DFT and QEpy</h1>
<center>
<br>
<table>
  <tr>
      <td><p><h1>Rutgers Team</h1></p><p><h2>(Bhaskar, Jessica, Valeria)</h2></p></td> 
      <td><img src="../figures/logos/run.png" width=300 height=300 /></td>
  </tr>
  <tr>
    <td></td>
  </tr>
</table>
<br>
<center>https://github.com/Quantum-MultiScale-Template/DFT-Intro/2025/QDMS/qepy/QDMS_2025_DFT-Intro.ipynb</center>

<br>

<center><h3> QDMS 2025 -- UCSD -- July 17, 2025</h3></center>
</div>


# Goals of this lecture + hands-on session
- Reasons for considering DFT
- Basics of the theory behind DFT and Kohn-Sham DFT
- KS equations
- Basis sets: plane waves, Gaussian-type orbitals
- Pseudopotentials (local part)
- Sample of KS-DFT simulations

# The Real World
<table>
    <tr>
      <td><h3>Photocatalyst</h3></td>
        <td><h3>Catalytic nanoparticles</h3></td>
  </tr>
  <tr>
      <td><img src="../figures/science/photocatalyst.png" height=500 /></td>
      <td><img src="../figures/science/catalyst.png" height=500 /></td>
  </tr>
    <tr>
        <td>Chem. Comm., 43, 6551 (2009)</td>
        <td>PCCP, 21, 15080 (2019)</td>
    </tr>
</table>   

# Available electronic structure methods
<br>
<center>
    <img src="../figures/science/electronic_structure.png" width=1600 />
</center>

# The Hohenberg and Kohn theorems
<br>
<br>
$$
\Psi_0 \leftrightarrow n(r) \leftrightarrow v_{eN}(r)
$$

Therefore $n(r)$, $v_{eN}(r)$ or $\Psi_0$ hold the same information. 

In particular:

$$
E \equiv E[\Psi_0] \equiv E[v_{eN}] \equiv E[n]
$$

DFT exploits the latter as follows:

$$
E[n] = T_s[n] + E_{H}[n]+E_{xc}[n]+E_{eN}[n]+E_{NN}
$$

The *famous* KS equations are:

$$
\big[ -\frac{1}{2}\nabla^2 + v_s(r) \big] \phi_k(r) = \varepsilon_k \phi_k(r) ~~\text{with}~~
v_s(r) = \frac{\delta (E_H+E_{xc})}{\delta n(r)}+v_{eN}(r)
$$

<br>
<br>
<br>
<center>
<span style="font-size:45pt;"><i>               n(r)</i></span>
</center>
<br>
<br>
<br>
<center>...the density determines everything...</center>

# The Self-Consistent Field Method

The KS equations $-\frac{1}{2}\nabla^2 \phi_i(r) + v_s(r)\phi_i(r) = \varepsilon_i\phi_i(r)$ feature $v_s(r)$ which depends on the density:
<br>
<br>
$$
v_s[n](r) = \frac{\delta E_{H}[n]}{\delta n(r)} + \frac{\delta E_{xc}[n]}{\delta n(r)} + v_{eN}(r)
$$

But the density depends on the KS orbitals, $\{\phi_i(r)\}$:
<br>
<br>
$$
n(r) = \sum_i n_i |\phi_i(r)|^2\,\, n_i \text{ are the occupation numbers.}
$$

<center>
<div class="alert alert-danger">Can the problem be solved?</div>
</center>

<center>
<img src="../figures/science/SCF_scheme.png" width=1200 \>
</center>

<center>
<div class="alert alert-success">Most times it converges :)</div>
</center>

# DFT functionals

<b>Pure functionals</b>, dependent on $n(r)$, $|\nabla n(r)|$, $\nabla^2 n(r)$ and or $\tau_{s}(r) = \frac{1}{2}\sum_i n_i |\nabla\phi_i(r)|^2$:
- LDA
- GGA (PBE)
- meta-GGA (SCAN)

<b>Hybrid functionals</b>, dependent on pure functionals and on HF exchange, $E_x^{HF}[\{\phi\}]$.
- PBE0
- B3LYP
- HSE

# Basis sets (discretization)

To be able to run simulations, we need to discretize the space in which the KS wavefunctions / densities live. We will consider plane waves (PW) and Gaussian-type orbitals (GTOs). The general idea is the following:
$$
\phi_k(r) = \sum_\mu^M C_\mu^k \chi_\mu(r) \text{, where } \chi_\mu \text{ are basis functions}
$$

<table border="1" style="width:100%; text-align:center;">
    <tr>
        <th></th>
        <th style="text-align:center;border-right: 2px solid red;padding: 40px;">GTOs</th>
        <th style="text-align:center;">PW</th>
    </tr>
    <tr>
        <th style="text-align:center;padding: 40px;">Definition</th>
        <td style="text-align:center; border-right: 2px solid red;">
            \( \chi_\mu(r) = x^{i_\mu}y^{j_\mu}z^{k_\mu} e^{-\frac{|r-R_\mu|^2}{2\sigma_\mu^2}} \)
        </td>
        <td style="text-align:center;">
            \( \chi_\mu(r) = \frac{1}{\sqrt{\Omega}} e^{i G_\mu \cdot r},\,\,G_\mu = \left(\frac{2\pi\, i_\mu}{a},\frac{2\pi\, j_\mu}{b},\frac{2\pi\, k_\mu}{c}\right)\)
        </td>
    </tr>
    <tr>
        <th style="text-align:center;padding: 40px;">Location</th>
        <td style="text-align:center;border-right: 2px solid red;">
            \( R_\mu \in \text{centers of atoms}\)
        </td>
        <td style="text-align:center;">
            Spread out throughout the simulation cell
        </td>
    </tr>
    <tr>
        <th style="text-align:center;padding: 40px;">Number of functions</th>
        <td style="text-align:center;border-right: 2px solid red;">
            \( \simeq 10 \) per atom
        </td>
        <td style="text-align:center;">
            \( \propto \Omega = a \cdot b \cdot c \), generally \( 10^4 \) to \( 10^6 \)
        </td>
    </tr>
    <tr>
        <th style="text-align:center;padding: 40px;">Handling eN potential</th>
        <td style="text-align:center;border-right: 2px solid red;">
            <center><img src="../figures/science/fullpot.png" alt="GTOs image" width=400 ></center>
        </td>
        <td style="text-align:center;">
            <center><img src="../figures/science/pseudopot.png" alt="PW image" width=400 ></center>
        </td>
    </tr>
</table>


# Pseudo potentials cannot incorporate core electrons

<table border="1" style="width:100%; text-align:center;">
    <tr>
        <th><center><img src="../figures/science/Electron-Configuration.jpg" alt="econf" width=400 ></center></th>
        <th><center><img src="../figures/science/pseudo_core.png" alt="econf" width=400 ></center></th>
    </tr>
</table>






# Pseudopotential

In [ ]:
!wget https://pseudopotentials.quantum-espresso.org/upf_files/C.pbe-n-rrkjus_psl.1.0.0.UPF
!cat C.pbe-n-rrkjus_psl.1.0.0.UPF

# Bloch theorem and k-point sampling

1) Solids have periodic potentials:
$$
v_{eN}(r+n\hat a) = v_{eN}(r),\,\, \text{where } \hat a \text{ is a lattice vector.}
$$

<center><img src="../figures/science/periodic_pot.png" width=1300 /></center>

2) Bloch theorem states that the group of translations comes with 3 quantum numbers $\vec k = (k_a, k_b, k_c)$ representing translations along the 3 lattice vectors. The wavefunctions will be labelled by $k$ (dropped $\vec k$ for a lighter notation):
$$
\phi_i \to \phi_{ik} = e^{ik\cdot r} u_{ik}(r), \,\, k\in \text{First Brillouin Zone (FBZ)}
$$


# Setting up QEpy and QE's input file for KS-DFT

In [3]:
from qepy.driver import Driver
from qepy.io import QEInput

In [4]:
IFrame(src="https://www.quantum-espresso.org/Doc/INPUT_PW.html",width=1250, height=600)

### QEpy's input file is a dictionary containing QE's input keywords

In [5]:
from dftpy.formats import download_files
additional_files = {'Al.pbe-nl-kjpaw_psl.1.0.0.UPF' : 'https://pseudopotentials.quantum-espresso.org/upf_files/Al.pbe-nl-kjpaw_psl.1.0.0.UPF'}
download_files(additional_files)

In [6]:
qe_options = {
    '&control': {
        'calculation': "'scf'",
        'pseudo_dir': "'./'",
    },
    '&system': {
        'ibrav' : 0,
        'degauss': 0.005,
        'ecutwfc': 30,
        'occupations': "'smearing'"
    },
    'atomic_species': ['Al  26.98 Al.pbe-nl-kjpaw_psl.1.0.0.UPF'],
    'k_points gamma': [],
}

In [23]:
#!cat Al.pbe-nl-kjpaw_psl.1.0.0.UPF

In [7]:
options = {
    '&electrons': {
        'mixing_beta': 0.5},
    'cell_parameters angstrom':[
        '0.     2.025  2.025',
        '2.025  0.     2.025',
        '2.025  2.025  0.   '],
    'atomic_positions crystal': ['Al    0.0  0.0  0.0'],
    'k_points automatic': ['6 6 6 1 1 1'],
}

# SCF Convergence

## $\bullet$ smearing?
## $\bullet$ mixing_beta?
## $\bullet$ k-points?

Let's first run QEpy and solve the SCF for our simple Al system:

In [8]:
qe_options = QEInput.update_options(options, qe_options=qe_options)

In [9]:
driver = Driver(qe_options=qe_options, logfile=True)
%timeit -n1 -r1 driver.scf() 

2.04 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


# Smearing

$$
n(r) = \sum_i n_i |\phi_i(r)|^2,\qquad \{n_i\} \text{ are the occupation numbers.}
$$

Consider the electronic DOS of Aluminum:

<center><img src="../figures/science/DOS_example.png" width=1000 /></center>

<center>
<div class="alert alert-danger">Not possible to use integer occupations. The SCF would never converge!</div>
</center>

# Types of smearing

<b>Gaussian</b>: $$n_i = \frac{1}{2}\text{Erfc}\left[\frac{|\varepsilon_i-\mu|}{\text{degauss}}\right]$$

<b>Fermi-Dirac</b>: $$n_i = \left[ e^{\frac{|\varepsilon_i-\mu|}{\text{degauss}}}+1\right]^{-1}$$

<b>Others</b>: To be used in specific situations (semicondictors, or others), see [m-p](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.40.3616) and [m-v](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.82.3296).

# Density mixing

Broyden, Pulay (DIIS), ..., are SCF convergence accelerators. They all share the same logic:

1) Consider a trial of densities $\{ n_i(r) \}$ which are outputs of the $i$-th SCF cycle.

2) The converged (SCF) density is given as linear combination of the trials, $n = \sum_i c_i n_i$.

3) The "best" coefficients at each SCF step are found minimizing the error vector with respect to the coefficients. The error in QE is given by $R = \sum_i c_i R_i$ which is given in terms of the density residue, $\delta n_i = n_i - n_{i-1}$, as $R_i = \int \frac{\delta n_i(r)\delta n_i(r')}{|r-r'|}drdr'$.

4) A new density is generated with the new coefficients and the trial vectors to be reinserted in the SCF loop.

# Example with QEpy

Let's run QEpy with default QE's Broyden mixing:

In [11]:
driver=Driver(qe_options=qe_options, iterative = True, logfile='tmp.out') 
for i in range(60):
    driver.diagonalize()
    driver.mix()
    converged = driver.check_convergence()
    print ('Iter: ',i,' - Conv: ', driver.get_scf_error())
    if converged : break
driver.calc_energy()

Iter:  0  - Conv:  0.0014366084520385561
Iter:  1  - Conv:  6.798902354116732e-05
Iter:  2  - Conv:  5.766546336460826e-07


-39.501958966487685

<center>
    <div class="alert alert-success">Remember: 3 iterations.</div>
</center>

Let's run QEpy with a simple linear mixing, $n \to \alpha \, n_\text{prev} + (1- \alpha) \, n$

In [12]:
from dftpy.functional import Hartree 
driver=Driver(qe_options=qe_options, iterative = True, logfile='tmp.1.out') 
rho = driver.get_density().copy()
alpha = 0.7
for i in range(20):
    driver.diagonalize()
    #
    rho_new = driver.get_density().copy()
    drho = driver.data2field(rho_new-rho)
    error = Hartree.compute(drho).energy
    nc = np.abs(drho).sum()*driver.get_volume()/drho.size
    print ('Iter: ',i,' - Conv: ', error, 'dN:', nc)
    if error < 1e-8:
        driver.end_scf()
        break
    #
    rho = (1-alpha)*rho + alpha * rho_new
    driver.set_density(rho)
driver.calc_energy()

Iter:  0  - Conv:  0.0028501966183629075 dN: 0.2996635794844198
Iter:  1  - Conv:  0.00019844441773826955 dN: 0.08055563624276127
Iter:  2  - Conv:  1.2528837689876972e-05 dN: 0.02145203906195234
Iter:  3  - Conv:  9.438404776540801e-07 dN: 0.006068315220637983
Iter:  4  - Conv:  7.162966840434261e-08 dN: 0.0017917262380760593
Iter:  5  - Conv:  5.07536092952215e-09 dN: 0.0004872146569315403


-39.501959115709354

<center>
    <div class="alert alert-danger">5 iterations...</div>
</center>

<center>
    <div class="alert alert-success">Density mixing <b>does</b> improve convergence.</div>
</center>

# K-point sampling example with QEpy

1) First, let's see how many orbitals (bands) we have.

In [13]:
print(f"Number of orbitals (bands) considered by QE: {driver.get_number_of_bands()}") 

Number of orbitals (bands) considered by QE: 6


2) Let's take a look at the irreducible k-points considered by QE.

In [14]:
driver.get_ibz_k_points()

array([[ 0.08333333,  0.08333333,  0.08333333],
       [ 0.08333333,  0.08333333,  0.25      ],
       [ 0.08333333,  0.08333333,  0.41666667],
       [ 0.08333333,  0.08333333, -0.41666667],
       [ 0.08333333,  0.08333333, -0.25      ],
       [ 0.08333333,  0.08333333, -0.08333333],
       [ 0.08333333,  0.25      ,  0.25      ],
       [ 0.08333333,  0.25      ,  0.41666667],
       [ 0.08333333,  0.25      , -0.41666667],
       [ 0.08333333,  0.25      , -0.25      ],
       [ 0.08333333,  0.25      , -0.08333333],
       [ 0.08333333,  0.41666667,  0.41666667],
       [ 0.08333333,  0.41666667, -0.41666667],
       [ 0.08333333,  0.41666667, -0.25      ],
       [ 0.08333333,  0.41666667, -0.08333333],
       [ 0.08333333, -0.41666667, -0.41666667],
       [ 0.08333333, -0.41666667, -0.25      ],
       [ 0.08333333, -0.25      , -0.25      ],
       [ 0.25      ,  0.25      ,  0.25      ],
       [ 0.25      ,  0.25      ,  0.41666667],
       [ 0.25      ,  0.25      , -0.416